## Feed forward basic

Basic equation with just one layer:

$$NN(X) = f( W\cdot X + b)$$
with $f(\cdot)$ a ReLU function:
$$
f(x) = \begin{cases}
    0, & \text{if } x < 0\\
    x, & \text{if } x \geq 0
\end{cases}
$$
and $W$ is a matrix of weights per neuron in the layer with dimension $(|N|_{n+1} \times |N|_n)$
Derivative given the entry:
$$
\frac{dNN_j(X)}{dX_i} = \frac{dNN_j}{df}\frac{df}{dX_i} = 
\begin{cases}
    0, & \text{if } x < 0\\
    W_{ji}, & \text{if } x \geq 0
\end{cases}
$$

Example:

* Given a neural network with 1 layer and 2 neurons in the layer we have:
$$
W = \begin{bmatrix}
    1 & 2 & 3\\
    3 & 4 & 6
\end{bmatrix} 
$$

$$
b = \begin{bmatrix}
    1 \\
    3 
\end{bmatrix} 
$$

$$
X = \begin{bmatrix}
    1 \\
    2 \\
    3
\end{bmatrix} 
$$

In [2]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-03-08 10:20:05.870044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def relu(X):
    """_summary_

    Args:
        X (_type_): _description_

    Returns:
        _type_: _description_
    """
    return tf.keras.activations.relu(X).numpy()

def relu_derivative(X, partial_x = 0, partial_j = 0):
    """_summary_

    Args:
        X (_type_): _description_

    Returns:
        _type_: _description_
    """
    return X[partial_j][partial_x]

In [13]:
input_layer = keras.Input(shape = (2,), name='input_nn')
dense_unit = layers.Dense(
    units = 2,
    activation = 'relu',
    name = 'dense_layer'
)(input_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[dense_unit],
    name = 'test_model'
)

In [14]:
# Obtener los pesos de la capa oculta
w = custom_model.get_layer('dense_layer').get_weights()[0].T
b = custom_model.get_layer('dense_layer').get_weights()[1].T

In [15]:
x = tf.convert_to_tensor(
    np.array(range(-7,-1)).reshape((2,3)),
    dtype = tf.float32    
)

In [16]:
hand_result = tf.transpose(relu(tf.matmul(w, x)))
model_output = custom_model(tf.transpose(x))

In [17]:
hand_result, model_output

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.       , 1.5790672],
        [0.       , 1.7795793],
        [0.       , 1.9800918]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.       , 1.5790672],
        [0.       , 1.7795793],
        [0.       , 1.9800918]], dtype=float32)>)

In [18]:
# Test 
xs = tf.Variable(x, trainable = False, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(tf.transpose(xs))
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})

In [19]:
grads['x']

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-0.79370534, -0.79370534, -0.79370534],
       [ 0.9942175 ,  0.9942175 ,  0.9942175 ]], dtype=float32)>

In [20]:
tf.linalg.diag_part(jacobian['x']),jacobian['x']

(<tf.Tensor: shape=(3, 2, 2), dtype=float32, numpy=
 array([[[ 0.        ,  0.        ],
         [-0.79370534,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.9942175 ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2, 2, 3), dtype=float32, numpy=
 array([[[[ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]],
 
         [[-0.79370534,  0.        ,  0.        ],
          [ 0.9942175 ,  0.        ,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]],
 
         [[ 0.        , -0.79370534,  0.        ],
          [ 0.        ,  0.9942175 ,  0.        ]]],
 
 
        [[[ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]],
 
         [[ 0.        ,  0.        , -0.79370534],
          [ 0.        ,  0.        ,  0.9942175 ]]]], dtype=floa

In [66]:
custom_grads = np.zeros((
    2,4
))
for j in range(2):
    for i in range(4):
        custom_grads[j,i] = relu_derivative(w, partial_x = i, partial_j = j)

In [67]:
custom_grads

array([[ 0.55118585, -0.1299386 , -0.6776278 , -0.80925226],
       [-0.80925632, -0.00675702, -0.05111217, -0.39438462]])

In [23]:
import tensorflow as tf

# Definir dos tensores
A = tf.constant([5, 3, 2])
B = tf.constant([1, 7, 9])

# Calcular el cuadrado de la diferencia entre los tensores
C = tf.math.squared_difference(A, B)

print(C)


tf.Tensor([16 16 49], shape=(3,), dtype=int32)


In [28]:
import tensorflow as tf

x1 = tf.Variable([[2.0, 3.0], [5, 8]], name = 'x1')
x2 = tf.constant([5.0, 8.0], name = 'x2')
with tf.GradientTape() as t:
    t.watch(x1)
    t.watch(x2)
    y = tf.math.add(tf.square(x1), tf.square(x2))

jacobian = t.gradient(y, {
    'x1':x1,
    'x2':x2
})
print(jacobian)

{'x1': <tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 4.,  6.],
       [10., 16.]], dtype=float32)>, 'x2': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([20., 32.], dtype=float32)>}
